In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.semi_supervised import LabelPropagation
from nltk.corpus import stopwords
from xgboost import XGBClassifier
import string
import nltk

In [3]:
# Certifique-se de baixar os recursos necessários do NLTK
nltk.download('stopwords')

# Carregar o CSV
df = pd.read_csv('./dados/pre-processed.csv')


# Converter os rótulos para valores numéricos
df['label'] = df['label'].apply(lambda x: 1 if x == 'fake' else 0)

# Função para limpar as manchetes (remoção de pontuação e stopwords)
def limpar_texto(texto):
    # Remover pontuações
    texto = ''.join([char for char in texto if char not in string.punctuation])
    # Converter para minúsculas
    texto = texto.lower()
    # Remover stopwords
    stop_words = set(stopwords.words('portuguese'))
    texto = ' '.join([word for word in texto.split() if word not in stop_words])
    return texto

# Aplicar a função de limpeza nas manchetes
df['manchete_limpa'] = df['preprocessed_news'].apply(limpar_texto)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cagol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df['label'].value_counts()

label
1    3600
0    3600
Name: count, dtype: int64

In [5]:
# Inicializar o vetorizador TF-IDF
tfidf = TfidfVectorizer()

# Aplicar o vetorizador nas manchetes limpas
X_tfidf = tfidf.fit_transform(df['manchete_limpa'])

In [6]:

# Separar a variável preditora (X) e a variável resposta (y)
X = X_tfidf  # A matriz TF-IDF é a nossa variável preditora
y = df['label']  # Coluna 'label' como variável resposta

# Dividir o dataset em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar o modelo de Regressão Logística
modelo = LogisticRegression()

# Treinar o modelo
modelo.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_log = modelo.predict(X_test)

# Avaliar o modelo
print("Acurácia do modelo:", accuracy_score(y_test, y_pred_log))
print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred_log))

# (Opcional) Salvar as previsões junto com o conjunto de teste para análise posterior
df_teste = pd.DataFrame({
    'manchete_original': df['preprocessed_news'].iloc[y_test.index],
    'label_real': y_test,
    'label_previsto': y_pred_log
})

Acurácia do modelo: 0.9465277777777777

Relatório de classificação:
              precision    recall  f1-score   support

           0       0.96      0.93      0.95       722
           1       0.94      0.96      0.95       718

    accuracy                           0.95      1440
   macro avg       0.95      0.95      0.95      1440
weighted avg       0.95      0.95      0.95      1440



In [7]:
# Inicializar e treinar o modelo XGBoost
modelo_xgb = XGBClassifier(eval_metric='logloss')
modelo_xgb.fit(X_train, y_train)

# Fazer previsões e avaliar o modelo XGBoost
y_pred_xgb = modelo_xgb.predict(X_test)
print("\nAcurácia do XGBoost:", accuracy_score(y_test, y_pred_xgb))
print("\nRelatório de classificação do XGBoost:")
print(classification_report(y_test, y_pred_xgb))
# Comparação das previsões e salvar resultados
df_comparacao = pd.DataFrame({
    'manchete_original': df['manchete_limpa'].iloc[y_test.index],
    'label_real': y_test,
    'label_pred_log': y_pred_log,
    'label_pred_xgb': y_pred_xgb
})
df_comparacao.to_csv('comparacao_modelos.csv', index=False)
print("Comparação dos modelos salva em 'comparacao_modelos.csv'.")


Acurácia do XGBoost: 0.9527777777777777

Relatório de classificação do XGBoost:
              precision    recall  f1-score   support

           0       0.94      0.97      0.95       722
           1       0.96      0.94      0.95       718

    accuracy                           0.95      1440
   macro avg       0.95      0.95      0.95      1440
weighted avg       0.95      0.95      0.95      1440

Comparação dos modelos salva em 'comparacao_modelos.csv'.
